# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

869

In [8]:
len(sales_lumpy)

231

In [ ]:
len(sales_erratic)

458

In [ ]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [9]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
64,HOBBIES_1_066_CA_1_validation,HOBBIES_1_066,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,4,0,0,0,0,0,0,6,0
126,HOBBIES_1_131_CA_1_validation,HOBBIES_1_131,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,1,0,0,1,0
135,HOBBIES_1_140_CA_1_validation,HOBBIES_1_140,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,0,0,0,0,0,1,0,4
140,HOBBIES_1_145_CA_1_validation,HOBBIES_1_145,HOBBIES_1,HOBBIES,CA_1,CA,0,12,0,4,...,0,11,0,0,1,3,0,12,0,2
162,HOBBIES_1_167_CA_1_validation,HOBBIES_1_167,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,HOBBIES_2_141_CA_4_validation,HOBBIES_2_141,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9704,HOBBIES_2_142_CA_4_validation,HOBBIES_2_142,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Croston algorithm

In [10]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [11]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [12]:
sales_pattern = sales_lumpy.copy()

In [13]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [14]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [15]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [16]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [17]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [18]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [19]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [20]:
len(sample_list_filtered)

123

# Hyperparameter tuning for Croston

In [21]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [22]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_2_059_CA_4
Progressing: 0.0 %
Currently Running: HOBBIES_2_107_CA_3
Progressing: 0.81 %
Currently Running: HOBBIES_1_167_CA_2
Progressing: 1.63 %
Currently Running: HOBBIES_2_145_CA_2
Progressing: 2.44 %
Currently Running: HOBBIES_2_007_CA_3
Progressing: 3.25 %
Currently Running: HOBBIES_1_234_CA_1
Progressing: 4.07 %
Currently Running: HOBBIES_2_144_CA_2
Progressing: 4.88 %
Currently Running: HOBBIES_2_057_CA_3
Progressing: 5.69 %
Currently Running: HOBBIES_1_167_CA_1
Progressing: 6.5 %
Currently Running: HOBBIES_2_131_CA_2
Progressing: 7.32 %
Currently Running: HOBBIES_2_102_CA_1
Progressing: 8.13 %
Currently Running: HOBBIES_2_019_CA_1
Progressing: 8.94 %
Currently Running: HOBBIES_2_130_CA_3
Progressing: 9.76 %
Currently Running: HOBBIES_2_060_CA_1
Progressing: 10.57 %
Currently Running: HOBBIES_2_085_CA_1
Progressing: 11.38 %
Currently Running: HOBBIES_2_064_CA_2
Progressing: 12.2 %
Currently Running: HOBBIES_2_046_CA_2
Progressing: 13.01 %
Currently Run

In [23]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOBBIES_2_059_CA_4,1.0,2.362223,0.05
1,HOBBIES_2_059_CA_4,1.0,2.047772,0.05
2,HOBBIES_2_059_CA_4,1.0,2.016646,0.05
3,HOBBIES_2_059_CA_4,2.0,1.985818,0.05
4,HOBBIES_2_059_CA_4,0.0,1.986257,0.05
...,...,...,...,...
17707,HOBBIES_2_140_CA_1,2.0,1.804584,0.90
17708,HOBBIES_2_140_CA_1,3.0,1.045940,0.90
17709,HOBBIES_2_140_CA_1,3.0,2.658053,0.90
17710,HOBBIES_2_140_CA_1,5.0,2.963033,0.90


# Summay all metrics

In [24]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [25]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.09700768215000011 minutes


In [26]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_2,0.05,0.786568,64.2096,69.2538,26.9926,0.561834,0.708066
1,HOBBIES_1_002_CA_2,0.10,0.651944,53.2199,63.5572,17.2641,0.465674,0.646639
2,HOBBIES_1_002_CA_2,0.15,0.635410,51.8702,62.4709,16.0159,0.453864,0.640946
3,HOBBIES_1_002_CA_2,0.20,0.642517,52.4504,62.2806,16.5240,0.458941,0.647505
4,HOBBIES_1_002_CA_2,0.25,0.640091,52.2523,61.6144,16.4253,0.457208,0.656419
...,...,...,...,...,...,...,...,...
2209,HOBBIES_2_149_CA_1,0.70,1.227831,280.6471,186.2039,61.8931,0.350809,0.378972
2210,HOBBIES_2_149_CA_1,0.75,1.318871,301.4563,184.8341,56.4626,0.376820,0.403708
2211,HOBBIES_2_149_CA_1,0.80,1.422725,325.1943,183.3953,50.2784,0.406493,0.436394
2212,HOBBIES_2_149_CA_1,0.85,1.548472,353.9364,181.8137,42.8351,0.442421,0.480799


# Check what products has unexpected metrics result

In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
630,HOBBIES_2_014_CA_1,0.05,inf,inf,200.0,NaN,0.069448,0.069448
631,HOBBIES_2_014_CA_1,0.10,inf,inf,200.0,NaN,0.078249,0.078249
632,HOBBIES_2_014_CA_1,0.15,inf,inf,200.0,NaN,0.089316,0.089316
633,HOBBIES_2_014_CA_1,0.20,inf,inf,200.0,NaN,0.103045,0.103045
634,HOBBIES_2_014_CA_1,0.25,inf,inf,200.0,NaN,0.119698,0.119698
...,...,...,...,...,...,...,...,...
1957,HOBBIES_2_131_CA_2,0.70,inf,inf,200.0,NaN,0.470597,0.470597
1958,HOBBIES_2_131_CA_2,0.75,inf,inf,200.0,NaN,0.492420,0.492420
1959,HOBBIES_2_131_CA_2,0.80,inf,inf,200.0,NaN,0.512688,0.512688
1960,HOBBIES_2_131_CA_2,0.85,inf,inf,200.0,NaN,0.528217,0.528217


In [28]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
630,HOBBIES_2_014_CA_1,0.05,inf,inf,200.0,NaN,0.069448,0.069448
631,HOBBIES_2_014_CA_1,0.10,inf,inf,200.0,NaN,0.078249,0.078249
632,HOBBIES_2_014_CA_1,0.15,inf,inf,200.0,NaN,0.089316,0.089316
633,HOBBIES_2_014_CA_1,0.20,inf,inf,200.0,NaN,0.103045,0.103045
634,HOBBIES_2_014_CA_1,0.25,inf,inf,200.0,NaN,0.119698,0.119698
...,...,...,...,...,...,...,...,...
1957,HOBBIES_2_131_CA_2,0.70,inf,inf,200.0,NaN,0.470597,0.470597
1958,HOBBIES_2_131_CA_2,0.75,inf,inf,200.0,NaN,0.492420,0.492420
1959,HOBBIES_2_131_CA_2,0.80,inf,inf,200.0,NaN,0.512688,0.512688
1960,HOBBIES_2_131_CA_2,0.85,inf,inf,200.0,NaN,0.528217,0.528217


In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
882,HOBBIES_2_035_CA_3,0.05,0.869697,99.3939,NaN,98.7566,0.372727,0.604820
883,HOBBIES_2_035_CA_3,0.10,0.869697,99.3939,NaN,98.7219,0.372727,0.604613
884,HOBBIES_2_035_CA_3,0.15,0.869697,99.3939,NaN,98.6832,0.372727,0.604383
885,HOBBIES_2_035_CA_3,0.20,0.869697,99.3939,NaN,98.6398,0.372727,0.604126
886,HOBBIES_2_035_CA_3,0.25,0.869697,99.3939,NaN,98.5907,0.372727,0.603835
887,HOBBIES_2_035_CA_3,0.30,0.869697,99.3939,NaN,98.5348,0.372727,0.603505
888,HOBBIES_2_035_CA_3,0.35,0.869697,99.3939,NaN,98.4706,0.372727,0.603127
889,HOBBIES_2_035_CA_3,0.40,0.869697,99.3939,NaN,98.3959,0.372727,0.602690
890,HOBBIES_2_035_CA_3,0.45,0.869697,99.3939,NaN,98.3082,0.372727,0.602178
891,HOBBIES_2_035_CA_3,0.50,0.869697,99.3939,NaN,98.2035,0.372727,0.601570


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
630,HOBBIES_2_014_CA_1,0.05,inf,inf,200.0,NaN,0.069448,0.069448
631,HOBBIES_2_014_CA_1,0.10,inf,inf,200.0,NaN,0.078249,0.078249
632,HOBBIES_2_014_CA_1,0.15,inf,inf,200.0,NaN,0.089316,0.089316
633,HOBBIES_2_014_CA_1,0.20,inf,inf,200.0,NaN,0.103045,0.103045
634,HOBBIES_2_014_CA_1,0.25,inf,inf,200.0,NaN,0.119698,0.119698
...,...,...,...,...,...,...,...,...
1957,HOBBIES_2_131_CA_2,0.70,inf,inf,200.0,NaN,0.470597,0.470597
1958,HOBBIES_2_131_CA_2,0.75,inf,inf,200.0,NaN,0.492420,0.492420
1959,HOBBIES_2_131_CA_2,0.80,inf,inf,200.0,NaN,0.512688,0.512688
1960,HOBBIES_2_131_CA_2,0.85,inf,inf,200.0,NaN,0.528217,0.528217


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [33]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [34]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 4.878048780487805 %
Percentage of unexpected values of WMAPE is: 4.878048780487805 %
Percentage of unexpected values of SMAPE is: 0.8130081300813009 %
Percentage of unexpected values of MAPE is: 4.878048780487805 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [35]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [36]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.012977,107.628760,120.594843,64.807904,2.126238,2.768268
0.10,0.941626,100.789078,117.292720,60.077302,2.043978,2.702768
0.15,0.913316,98.639729,115.081358,59.177303,2.026696,2.675839
0.20,0.907141,98.401082,113.950197,59.939192,2.024452,2.652599
0.25,0.908727,98.870376,113.333968,61.093446,2.037283,2.645806
0.30,0.914775,99.777021,113.210128,62.737338,2.065447,2.651034
0.35,0.924614,100.932321,113.428806,64.708197,2.100364,2.666329
0.40,0.934231,102.029884,113.617587,66.501503,2.131941,2.690925
0.45,0.945052,103.205572,113.885232,68.265908,2.170692,2.723538


In [37]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.2
The optimum Alpha based on Mean SMAPE is: 0.3
The optimum Alpha based on Mean MAPE is: 0.15
The optimum Alpha based on Mean MAE is: 0.2
The optimum Alpha based on Mean RMSE is: 0.25


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.062117,72.15745,86.76505,61.24290,3.114458,3.851939
0.10,0.998497,67.98890,75.80255,55.38315,2.632921,3.440790
0.15,0.938767,66.25455,73.33200,56.46700,2.630586,3.414761
0.20,0.903654,65.49920,71.27225,57.10015,2.606581,3.143779
0.25,0.882766,64.06840,69.87680,57.02665,2.553123,3.132278
0.30,0.843319,62.67790,67.56535,57.09930,2.536494,3.178992
0.35,0.839414,62.89210,66.76190,58.07205,2.524637,3.249838
0.40,0.853301,61.06010,66.87810,57.81635,2.595742,3.278589
0.45,0.861317,62.01375,67.49110,57.26275,2.609716,3.329361


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.35
The optimum Alpha based on Median of WMAPE is: 0.4
The optimum Alpha based on Median of SMAPE is: 0.35
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.65
The optimum Alpha based on Median of RMSE is: 0.25


# Run Croston model after decide best paramaters

In [38]:
best_alpha = 0.15

In [39]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_002_CA_2,HOBBIES_1_017_CA_4,HOBBIES_1_027_CA_4,HOBBIES_1_042_CA_2,HOBBIES_1_053_CA_3,HOBBIES_1_053_CA_4,HOBBIES_1_066_CA_1,HOBBIES_1_066_CA_2,HOBBIES_1_066_CA_3,...,HOBBIES_2_145_CA_3,HOBBIES_2_147_CA_4,HOBBIES_2_148_CA_1,HOBBIES_2_148_CA_2,HOBBIES_2_148_CA_3,HOBBIES_2_148_CA_4,HOBBIES_2_149_CA_1,HOBBIES_2_149_CA_2,HOBBIES_2_149_CA_3,HOBBIES_2_149_CA_4
0,11101,2.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0
1,11102,1.0,1.0,0.0,2.0,0.0,6.0,0.0,0.0,0.0,...,5.0,0.0,1.0,1.0,1.0,1.0,3.0,6.0,0.0,7.0
2,11103,2.0,4.0,0.0,0.0,0.0,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,7.0,2.0,0.0,1.0,1.0,0.0,1.0
3,11104,1.0,2.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,2.0,1.0,4.0,1.0,1.0,0.0,3.0,0.0
4,11105,0.0,0.0,0.0,2.0,6.0,1.0,0.0,0.0,1.0,...,0.0,0.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,1.0,5.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
269,11609,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0
270,11610,1.0,1.0,0.0,2.0,0.0,4.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,2.0,1.0
271,11611,2.0,2.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0


In [40]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [41]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [42]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_066_CA_1,0.0,0.117603
1,HOBBIES_1_066_CA_1,3.0,0.117603
2,HOBBIES_1_131_CA_1,20.0,8.632702
3,HOBBIES_1_131_CA_1,8.0,10.091769
4,HOBBIES_1_140_CA_1,0.0,1.242724
...,...,...,...
457,HOBBIES_2_147_CA_4,0.0,0.000000
458,HOBBIES_2_148_CA_4,0.0,0.648370
459,HOBBIES_2_148_CA_4,0.0,0.648370
460,HOBBIES_2_149_CA_4,1.0,0.083723


In [43]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [44]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_2,0.526764,105.3528,134.8812,51.7213,1.053528,1.053702
1,HOBBIES_1_017_CA_4,0.508450,101.6901,109.1278,16.7286,1.016901,1.224606
2,HOBBIES_1_027_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOBBIES_1_042_CA_2,inf,37.5388,46.2555,37.5388,0.750777,0.751999
4,HOBBIES_1_053_CA_3,inf,inf,200.0000,NaN,0.825410,0.825410
...,...,...,...,...,...,...,...
226,HOBBIES_2_148_CA_4,inf,inf,200.0000,NaN,0.648370,0.648370
227,HOBBIES_2_149_CA_1,1.416863,94.4575,175.8341,93.4911,1.416863,1.506175
228,HOBBIES_2_149_CA_2,inf,inf,200.0000,NaN,0.241600,0.241600
229,HOBBIES_2_149_CA_3,0.500000,100.0000,184.8396,91.7981,0.500000,0.651696


In [45]:
df_result_final.isnull().sum()

Product     0
MASE       37
WMAPE      37
SMAPE      37
MAPE       89
MAE         0
RMSE        0
dtype: int64

In [46]:
df_result_final.MAPE.mean()

58.29052605633804

In [47]:
df_result_final.to_csv('Croston_Lumpy_Metrics.csv')